In [1]:
from pymongo import MongoClient
import pandas as pd
import json
from pathlib import Path

def conectar_mongodb():
    client = MongoClient("mongodb://127.0.0.1:27017/")
    db = client['statistics_futebol']
    return db['futebol']

def carregar_csv():
    datasets = {
        'cartoes': pd.read_csv('csv/campeonato-brasileiro-cartoes.csv'),
        'estatisticas': pd.read_csv('csv/campeonato-brasileiro-estatisticas-full.csv'),
        'brasileirao': pd.read_csv('csv/campeonato-brasileiro-full.csv'),
        'gols': pd.read_csv('csv/campeonato-brasileiro-gols.csv')
    }
    return datasets

def carregar_json():
    dados_json = {}
    for ano in range(2003, 2024):  # Atualizando para incluir todos os anos até 2023
        try:
            arquivo = f'data/brasileirao-{ano}.json'
            with open(arquivo, 'r', encoding='utf-8') as f:
                dados_json[ano] = json.load(f)
        except FileNotFoundError:
            print(f"Arquivo do ano {ano} não encontrado")
    return dados_json

def inserir_dados(collection, datasets, dados_json):
    # Limpa a coleção antes de inserir novos dados
    collection.delete_many({})
    
    # Inserindo dados CSV
    for nome, df in datasets.items():
        documentos = df.to_dict('records')
        for doc in documentos:
            doc['tipo'] = nome  # Adiciona um campo para identificar o tipo de dado
        collection.insert_many(documentos)
    
    # Inserindo dados JSON
    for ano, dados in dados_json.items():
        collection.insert_one({
            'tipo': 'json',
            'ano': ano,
            'dados': dados
        })

def main():
    # Conectar ao MongoDB
    collection = conectar_mongodb()
    
    # Carregar dados
    datasets = carregar_csv()
    dados_json = carregar_json()
    
    # Inserir dados
    inserir_dados(collection, datasets, dados_json)
    
    # Verificar inserção
    total_documentos = collection.count_documents({})
    print(f"Total de documentos inseridos: {total_documentos}")
    
    # Mostrar contagem por tipo
    for tipo in ['cartoes', 'estatisticas', 'brasileirao', 'gols', 'json']:
        count = collection.count_documents({'tipo': tipo})
        print(f"Documentos do tipo {tipo}: {count}")

if __name__ == "__main__":
    main()

Total de documentos inseridos: 53025
Documentos do tipo cartoes: 18857
Documentos do tipo estatisticas: 16810
Documentos do tipo brasileirao: 8405
Documentos do tipo gols: 8932
Documentos do tipo json: 21
